In [6]:
import cv2
import numpy as np
import glob
import os

def calibrate_camera(calib_folder, chessboard_size=(7, 9), square_size=15.0):
    # Prepare object points
    objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
    objp *= square_size

    objpoints = []  # 3D points in real world
    imgpoints = []  # 2D points in image plane

    # Load all calibration images in the folder that match "image (*.jpg)"
    image_paths = glob.glob(os.path.join(calib_folder, "Image*.jpeg"))
    image_paths.sort()  # Optional: sort the files in order

    if not image_paths:
        print(f"❌ No calibration images found in {calib_folder}")
        return None, None

    for fname in image_paths:
        img = cv2.imread(fname)
        if img is None:
            print(f"❌ Could not read {fname}")
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find chessboard corners
        ret, corners = cv2.findChessboardCorners(
            gray, chessboard_size,
            flags=cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE
        )

        if ret:
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(
                gray, corners, (11, 11), (-1, -1),
                criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
            )
            imgpoints.append(corners2)
            print(f"✅ Found chessboard in {os.path.basename(fname)}")
        else:
            print(f"❌ Chessboard NOT found in {os.path.basename(fname)}")

    if not objpoints:
        raise RuntimeError("No valid chessboards were detected. Calibration failed.")

    # Calibrate
    ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, gray.shape[::-1], None, None)

    print("\n🎯 Calibration Results:")
    print("Camera matrix:\n", camera_matrix)
    print("Distortion coefficients:\n", dist_coeffs.ravel())

    return camera_matrix, dist_coeffs

def main():
    calib_folder = "cal"  # Folder name with your images
    calibrate_camera(calib_folder, chessboard_size=(7, 9), square_size=15.0)

if __name__ == "__main__":
    main()


✅ Found chessboard in Image (1).jpeg
✅ Found chessboard in Image (10).jpeg
❌ Chessboard NOT found in Image (11).jpeg
✅ Found chessboard in Image (12).jpeg
✅ Found chessboard in Image (13).jpeg
✅ Found chessboard in Image (14).jpeg
✅ Found chessboard in Image (15).jpeg
✅ Found chessboard in Image (16).jpeg
✅ Found chessboard in Image (17).jpeg
✅ Found chessboard in Image (18).jpeg
✅ Found chessboard in Image (19).jpeg
❌ Chessboard NOT found in Image (2).jpeg
✅ Found chessboard in Image (20).jpeg
✅ Found chessboard in Image (3).jpeg
✅ Found chessboard in Image (4).jpeg
✅ Found chessboard in Image (5).jpeg
✅ Found chessboard in Image (6).jpeg
✅ Found chessboard in Image (7).jpeg
✅ Found chessboard in Image (8).jpeg
✅ Found chessboard in Image (9).jpeg
✅ Found chessboard in Image.jpeg

🎯 Calibration Results:
Camera matrix:
 [[2.68010484e+03 0.00000000e+00 1.20555833e+03]
 [0.00000000e+00 2.69887481e+03 1.60406955e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients